In [76]:
%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.plot import *
from hossam.tensor import *
from pandas import read_excel, pivot_table, melt, to_datetime
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from scipy.stats import linregress

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
origin = read_excel("20년도 문자내역.xlsx")
origin

,날짜,제목,내용,충전(충전금),충전(포인트),충전(씨앗),사용(충전금),사용(포인트),사용(씨앗),잔액(충전금),잔액(포인트),잔액(씨앗),상세내용
0,2020-12-28,[제임스짐휘트니스 임시휴관 안,장문(일반),0,0,0,-19200,0,-3000,80832,0,NaN,NaN
1,2020-12-28,NaN,씨앗구매,0,0,3000,0,0,0,100032,0,NaN,문자씨앗 3?000건
2,2020-12-28,NaN,씨앗 구매로 인한 충전금 차감,0,0,0,-51000,0,0,100032,0,NaN,문자씨앗 3?000건
3,2020-12-28,[제임스짐휘트니스 임시휴관 안,장문(일반),0,0,0,-50,0,0,151032,0,NaN,NaN
4,2020-12-24,NaN,충전(가상계좌),100000,0,0,0,0,0,151082,0,NaN,가상계좌입금(VA)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,2020-01-15,NaN,씨앗 구매로 인한 충전금 차감,0,0,0,-51000,0,0,75181,0,NaN,문자씨앗 3?000건
173,2020-01-13,NaN,충전(신용카드),100000,0,0,0,0,0,126181,0,NaN,신용카드입금(C)
174,2020-01-13,문자씨앗 적립,룰렛당첨,0,0,3,0,0,0,26181,0,NaN,룰렛이벤트 문자씨앗 적립
175,2020-01-13,문자씨앗 적립,기타,0,0,1000,0,0,0,26181,0,NaN,포인트교환(16?000뿌리)


In [78]:
df1 = origin[['날짜', '제목']]
df1

,날짜,제목
0,2020-12-28,[제임스짐휘트니스 임시휴관 안
1,2020-12-28,NaN
2,2020-12-28,NaN
3,2020-12-28,[제임스짐휘트니스 임시휴관 안
4,2020-12-24,NaN
...,...,...
172,2020-01-15,NaN
173,2020-01-13,NaN
174,2020-01-13,문자씨앗 적립
175,2020-01-13,문자씨앗 적립


In [79]:
# 연월만 있는 새로운 열 생성
df1['연월'] = df1['날짜'].dt.to_period('M')
df1 = df1.drop('날짜',axis=1)
df1

,제목,연월
0,[제임스짐휘트니스 임시휴관 안,2020-12
1,NaN,2020-12
2,NaN,2020-12
3,[제임스짐휘트니스 임시휴관 안,2020-12
4,NaN,2020-12
...,...,...
172,NaN,2020-01
173,NaN,2020-01
174,문자씨앗 적립,2020-01
175,문자씨앗 적립,2020-01


In [80]:
df1.dropna(subset=['제목'], inplace=True)
df2 = df1[df1['제목'].str.contains('EVENT|주년|여름')]

# 결과 확인
print(df2)

                             제목       연월
22        [ !! 7주년 기념 리워드 혜택 !!  2020-11
24        [ !! 7주년 기념 리워드 혜택 !!  2020-11
27            [제임스짐 휘트니스 7주년 기념  2020-11
28           [ 제임스짐 휘트니스 7주년 리워  2020-11
86   [ Jamesgym Fitness ]  ★여름한  2020-06
89           [ 제임스짐 휘트니스 6주년 기념  2020-06
122   ★ Early Bird EVENT  ★  재등  2020-03
142       [ 제임스짐 휘트니스 EVENT ! ]  2020-02
143       [ 제임스짐 휘트니스 EVENT ! ]  2020-02
144       [ 제임스짐 휘트니스 EVENT ! ]  2020-02
145       [ 제임스짐 휘트니스 EVENT ! ]  2020-02
146       [ 제임스짐 휘트니스 EVENT ! ]  2020-02
147       [ 제임스짐 휘트니스 EVENT ! ]  2020-02
167          [ 제임스짐 휘트니스 6주년 기념  2020-01
170          [제임스짐 휘트니스 6주년 기념]  2020-01


In [81]:
df2 = df2.drop_duplicates('연월')
df2

,제목,연월
22,[ !! 7주년 기념 리워드 혜택 !!,2020-11
86,[ Jamesgym Fitness ] ★여름한,2020-06
122,★ Early Bird EVENT ★ 재등,2020-03
142,[ 제임스짐 휘트니스 EVENT ! ],2020-02
167,[ 제임스짐 휘트니스 6주년 기념,2020-01


In [82]:
df2['광고'] = 1

In [83]:
df2.drop('제목',axis=1)

,연월,광고
22,2020-11,1
86,2020-06,1
122,2020-03,1
142,2020-02,1
167,2020-01,1


In [84]:
origin = read_excel("21년도 문자내역.xlsx")
origin = origin[['날짜', '제목']]
origin['연월'] = origin['날짜'].dt.to_period('M')
origin.drop('날짜', axis=1, inplace=True)
origin

,제목,연월
0,[ 제임스짐 휘트니스 1월 휴관일,2021-12
1,NaN,2021-12
2,NaN,2021-12
3,[ 제임스짐 휘트니스 1월 휴관일,2021-12
4,[ 사회적거리두기 운영관련안내,2021-12
...,...,...
86,NaN,2021-01
87,[제임스짐 임시휴관 추가연장안,2021-01
88,문자씨앗 적립,2021-01
89,NaN,2021-01


In [85]:
origin.dropna(subset=['제목'], inplace=True)

In [86]:
df3 = origin[origin['제목'].str.contains('재등록|주년')]

In [87]:
df3['광고'] = 1

In [88]:
df3 = df3.drop('제목',axis=1)
df3

,연월,광고
17,2021-11,1
18,2021-11,1
59,2021-06,1


In [89]:
df3 = df3.drop_duplicates('연월')
df3

,연월,광고
17,2021-11,1
59,2021-06,1


In [90]:
origin = read_excel("22년도 문자내역.xlsx")
origin = origin[['날짜', '제목']]
origin['연월'] = origin['날짜'].dt.to_period('M')
origin.drop('날짜', axis=1, inplace=True)
origin.dropna(subset=['제목'], inplace=True)
origin

,제목,연월
0,[ JAMESGYM FITNESS ] [ 리모델,2022-12
1,[ JAMESGYM FITNESS ] 회원님,2022-12
2,[ 제임스짐 휘트니스 ] ▶재등,2022-11
3,[제임스짐 휘트니스] ◆ 2022,2022-11
11,[제임스짐 휘트니스],2022-11
12,▶재등록 우대가 마감임박◀ 장,2022-11
13,▶재등록 우대가 마감임박◀ 장,2022-11
14,[제임스짐 휘트니스] ◆ 2022 리,2022-11
15,[제임스짐 휘트니스] [2022 리,2022-11
16,문자씨앗 적립,2022-11


In [91]:
origin['제목'].unique()

array(['[ JAMESGYM FITNESS ]  [ 리모델', '[ JAMESGYM FITNESS ]  회원님',
       '[ 제임스짐 휘트니스 ]  ▶재등', '[제임스짐 휘트니스]  ◆ 2022', '[제임스짐 휘트니스]',
       '▶재등록 우대가 마감임박◀ 장', '[제임스짐 휘트니스] ◆ 2022 리', '[제임스짐 휘트니스]  [2022 리',
       '문자씨앗 적립', '■ 제임스짐 휘트니스 ALL리모"', '[안내 및 공지]   ■ 제임스짐', 'ㅇㄹㄴㅇㄹ',
       '[제임스짐 냉/온탕 사용안내] 안', '[ 제임스짐 휘트니스 ]  안녕하',
       '[ Jamesgym Fitness ]  Tip: 만', '[ Jamesgym Fitness ]  안녕하세',
       '[ JamesGym Fitness ]  ★ 2022', '안녕하세요 제임스짐 휘트니스',
       '[ JamesGym Fitness ]  여름을', '[ James Gym ]  회원님의 라커짐',
       '[ JamesGym Fitness ]  코로나', '[ JamesGym Fitness ]  안녕하세',
       '[ JamesGym Fitness ]   안녕하', '안녕하세요  제임스짐 휘트니스',
       '[ JAMESGYM FITNESS ]     ★ He', '[ JAMESGYM FITNESS ]  안녕하세',
       '제목 없음', '[ JAMESGYM FITNESS ]     안녕',
       '[ JAMESGYM FiTNESS EVENT ]', '[ JamesGym ] 회원님의 라커짐을',
       'SMS Replace 보내기', '[ JamesGym Fitness ]  [*이름*]',
       '[필라테스 NEW 할인 프로모션!]', '[필라테스 NEW 할인프로모션!]',
       '[ JamesGym Fitness ] ★ 2022 N', "★ 2022 '시ː작' 할인 프로모션",
  

In [92]:
df4 = origin[origin['제목'].str.contains('2022|프로모션|여름')]
df4['광고'] = 1
df4 = df4.drop('제목',axis=1)
df4

,연월,광고
3,2022-11,1
14,2022-11,1
15,2022-11,1
30,2022-06,1
34,2022-05,1
58,2022-02,1
60,2022-02,1
61,2022-02,1
62,2022-02,1
63,2022-01,1


In [93]:
df4 = df4.drop_duplicates('연월')
df4

,연월,광고
3,2022-11,1
30,2022-06,1
34,2022-05,1
58,2022-02,1
63,2022-01,1


In [94]:
origin = read_excel("23년도 문자내역.xlsx")
origin = origin[['날짜', '제목']]
origin['연월'] = origin['날짜'].dt.to_period('M')
origin.drop('날짜', axis=1, inplace=True)
origin.dropna(subset=['제목'], inplace=True)
origin

,제목,연월
1,[ Jamesgym Fitness ]기존 제,2023-12
4,[ Jamesgym Fitness ]안녕하세,2023-12
5,[ Jamesgym Fitness ]안녕하세,2023-12
6,[ Jamesgym Fitness ]안녕하세,2023-12
7,[ James Gym Fitness ]안녕하,2023-12
8,안녕하세요 제임스짐 휘트니스입,2023-12
9,[ JAMESGYM GOLF ] 안녕하세,2023-11
10,안녕하세요 제임스짐 휘트니스입,2023-10
11,안녕하세요 제임스짐 휘트니스,2023-10
12,제임스짐-레슨알,2023-08


In [95]:
origin['제목'].unique()

array(['[ Jamesgym Fitness ]기존 제', '[ Jamesgym Fitness ]안녕하세',
       '[ James Gym Fitness ]안녕하', '안녕하세요 제임스짐 휘트니스입',
       '[ JAMESGYM GOLF ]  안녕하세', '안녕하세요 제임스짐 휘트니스 ', '제임스짐-레슨알',
       '레슨 알림-제임스짐 휘트니스', '[ NEW FACE PT 프로모션 ]부담', '말복 ',
       '[ Jamesgym Fitness ]  안녕하세', '[ JamesGYM Fitness ]  안녕하세',
       '[JamesGym Fitness]  안녕하세요', '[ 제임스짐 휘트니스 ]', '[제임스짐 휘트니스]',
       '[제임스짐 휘트니스 긴급휴관공', '[ 제임스짐휘트니스 운영안내 ]', '[ 제임스짐휘트니스 앱 설치 및',
       '[ 제임스짐 삼성점 공지사항 ]', '제임스짐 삼성점 정식오픈일자', '제임스짐 삼성점 공식오픈일자',
       '●제임스짐 PT사전등록 이벤트●', '[ 제임스짐 휘트니스 ]  안녕하', '[ 제임스짐 휘트니스 ] 안녕하세',
       '[ 제임스짐 휘트니스 삼성점 ]', '[ 제임스짐  10주년 리모델링 기', '[ 제임스짐 삼성점은 10주년 리'],
      dtype=object)

In [96]:
df5 = origin[origin['제목'].str.contains('주년|등록|프로모션')]
df5['광고'] = 1
df5 = df5.drop('제목',axis=1)
df5 = df5.drop_duplicates('연월')
df5

,연월,광고
14,2023-08,1
54,2023-02,1
76,2023-01,1


In [97]:
total = [df2, df3, df4, df5]
total_data = pd.concat(total)
total_data

,제목,연월,광고
22,[ !! 7주년 기념 리워드 혜택 !!,2020-11,1
86,[ Jamesgym Fitness ] ★여름한,2020-06,1
122,★ Early Bird EVENT ★ 재등,2020-03,1
142,[ 제임스짐 휘트니스 EVENT ! ],2020-02,1
167,[ 제임스짐 휘트니스 6주년 기념,2020-01,1
17,NaN,2021-11,1
59,NaN,2021-06,1
3,NaN,2022-11,1
30,NaN,2022-06,1
34,NaN,2022-05,1


In [99]:
total_data = total_data.drop('제목',axis=1)
total_data

,연월,광고
22,2020-11,1
86,2020-06,1
122,2020-03,1
142,2020-02,1
167,2020-01,1
17,2021-11,1
59,2021-06,1
3,2022-11,1
30,2022-06,1
34,2022-05,1


In [101]:
total_data.to_excel('문자발송.xlsx',index=False)